In [2]:
!conda env list

# conda environments:
#
base                  *  F:\Program_Files\Anaconda
py3.6.3                  F:\Program_Files\Anaconda\envs\py3.6.3
vlfm                     F:\Program_Files\Anaconda\envs\vlfm



In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.neighbors import kneighbors_graph
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
import dgl
from models.GCN import GCN
import xgboost as xgb
from models.RNN import RNN
from models.SVM import SVM
from models.SVM import SVM_impl
from models.DecisionTree import DecisionTreeID3, DecisionTreeCART
from models.LogisticRegression import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [4]:
# 使用KNN图构建
def create_graph_from_data(features, n_neighbors=15):  # 增加邻居数
    knn_graph = kneighbors_graph(features, n_neighbors=n_neighbors, mode='connectivity', include_self=False)
    knn_graph = knn_graph.astype(np.float32).todense()

    # 转换为DGL图
    src, dst = np.where(knn_graph > 0)  # 获取非零元素的索引
    src = torch.tensor(src, dtype=torch.int64)
    dst = torch.tensor(dst, dtype=torch.int64)

    # 创建DGL图
    g = dgl.graph((src, dst))
    g.ndata['feat'] = torch.tensor(features, dtype=torch.float32)

    # 计算节点度数并进行归一化
    in_degrees = g.in_degrees().float()
    norm = 1.0 / in_degrees
    g.ndata['norm'] = norm

    return g

In [5]:
# 读取数据
mushroom_data = pd.read_csv('data/mushrooms.csv', dtype=str)

In [6]:
# 分离目标和特征
target = mushroom_data['class']
inputs = mushroom_data.drop(['class'], axis=1)

In [7]:
# 数据集划分
X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=0.2, random_state=24, stratify=target)

In [8]:
# 编码
enc_i = OrdinalEncoder()
enc_t = LabelEncoder()

x_train_transf = enc_i.fit_transform(X_train)
x_test_transf = enc_i.transform(X_test)

y_train_transf = enc_t.fit_transform(y_train)
y_test_transf = enc_t.transform(y_test)

In [9]:
# 确定分类任务的类别数
num_classes = len(np.unique(y_train_transf))

In [10]:
# 构建训练图
X_train_tensor = torch.tensor(x_train_transf, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_transf, dtype=torch.long)
X_test_tensor = torch.tensor(x_test_transf, dtype=torch.float32)

# 数据标准化
X_train_tensor = F.normalize(X_train_tensor, p=2, dim=1)
X_test_tensor = F.normalize(X_test_tensor, p=2, dim=1)

In [11]:
# 标准化数据
scaler = StandardScaler()
x_train_transf_scaler = scaler.fit_transform(x_train_transf)
x_test_transf_scaler = scaler.transform(x_test_transf)

In [12]:
# 初始化并训练Logistic Regression模型
log_reg_model = LogisticRegression(learning_rate=0.01, n_iter=1000, verbose=True, early_stopping=True)
log_reg_model.fit(x_train_transf_scaler, y_train_transf)

# 预测与评估
log_reg_predictions = log_reg_model.predict(x_test_transf_scaler)
accuracy_log_reg = accuracy_score(y_test_transf, log_reg_predictions)

# 计算训练集准确率
train_predictions = log_reg_model.predict(x_train_transf_scaler)
train_accuracy = accuracy_score(y_train_transf, train_predictions)
print(f'Training Accuracy (Logistic Regression): {train_accuracy * 100:.2f}%')

# 输出测试准确率
print(f'Test Accuracy (Logistic Regression): {accuracy_log_reg * 100:.2f}%')

Iteration 0: Loss = 0.6931
Iteration 100: Loss = 0.4415
Iteration 200: Loss = 0.3561


Iteration 300: Loss = 0.3149
Iteration 400: Loss = 0.2906
Iteration 500: Loss = 0.2745
Early stopping at iteration 597, Loss = 0.2633
Training Accuracy (Logistic Regression): 90.71%
Test Accuracy (Logistic Regression): 89.78%


In [ ]:
# 初始化并训练SVM模型
svm_model = SVM(C=1.0, kernel='linear', max_iter=1000)
svm_model.fit(x_train_transf_scaler, y_train_transf)

# 预测与评估
svm_predictions = svm_model.predict(x_test_transf_scaler)
accuracy_svm = accuracy_score(y_test_transf, svm_predictions)

# 输出测试准确率
print(f'Test Accuracy (SVM): {accuracy_svm * 100:.2f}%')

In [ ]:
# 定义超参数
input_size = X_train_tensor.shape[1]
hidden_size = 256
output_size = num_classes
num_layers = 2
learning_rate = 0.01
num_epochs = 100

# 初始化RNN模型
rnn_model = RNN(input_size, hidden_size, output_size, num_layers)
rnn_model = rnn_model.to(X_train_tensor.device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn_model.parameters(), lr=learning_rate)

# 训练RNN模型
for epoch in range(num_epochs):
    rnn_model.train()
    outputs = rnn_model(X_train_tensor.unsqueeze(1))  # 增加维度以匹配RNN输入
    loss = criterion(outputs, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 测试RNN模型
rnn_model.eval()
with torch.no_grad():
    test_outputs = rnn_model(X_test_tensor.unsqueeze(1))  # 增加维度以匹配RNN输入
    _, predicted = torch.max(test_outputs.data, 1)
    accuracy = accuracy_score(y_test_transf, predicted.cpu().numpy())
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch [10/100], Loss: 0.5170
Epoch [20/100], Loss: 0.3298
Epoch [30/100], Loss: 0.2514
Epoch [40/100], Loss: 0.2223
Epoch [50/100], Loss: 0.2029
Epoch [60/100], Loss: 0.1871
Epoch [70/100], Loss: 0.2490
Epoch [80/100], Loss: 0.1728
Epoch [90/100], Loss: 0.1689
Epoch [100/100], Loss: 0.1615
Test Accuracy: 94.65%


In [ ]:
# 使用ID3算法训练决策树
id3_model = DecisionTreeID3(max_depth=5)  # 可以根据需要调整最大深度
id3_model.fit(x_train_transf, y_train_transf)

# 在测试集上进行预测
id3_predictions = id3_model.predict(x_test_transf)

# 计算准确率
id3_accuracy = accuracy_score(y_test_transf, id3_predictions)
print(f'Test Accuracy (ID3): {id3_accuracy * 100:.2f}%')

Test Accuracy (ID3): 97.66%


In [ ]:
# 使用CART算法训练决策树
cart_model = DecisionTreeCART(max_depth=5)  # 可以根据需要调整最大深度
cart_model.fit(x_train_transf, y_train_transf)

# 在测试集上进行预测
cart_predictions = cart_model.predict(x_test_transf)

# 计算准确率
cart_accuracy = accuracy_score(y_test_transf, cart_predictions)
print(f'Test Accuracy (CART): {cart_accuracy * 100:.2f}%')

Test Accuracy (CART): 97.54%


In [ ]:
# XGBoost训练
dtrain = xgb.DMatrix(x_train_transf, label=y_train_transf)
dtest = xgb.DMatrix(x_test_transf, label=y_test_transf)

# XGBoost超参数设置
params = {
    'objective': 'binary:logistic',  # 二分类
    'eval_metric': 'logloss',
    'max_depth': 6,  # 树的最大深度
    'eta': 0.1,  # 学习率
    'subsample': 0.8,  # 子样本比例
    'colsample_bytree': 0.8,  # 树的列采样
    'n_jobs': 4
}

# 训练
num_round = 1000
bst = xgb.train(params, dtrain, num_round)

# 预测
y_pred_xgb = bst.predict(dtest)
y_pred_xgb = (y_pred_xgb > 0.5).astype(int)  # 二分类，阈值设置为0.5

# 计算准确率
accuracy_xgb = accuracy_score(y_test_transf, y_pred_xgb)
print(f'Test Accuracy (XGBoost): {accuracy_xgb * 100:.2f}%')

NameError: name 'xgb' is not defined

In [ ]:
G_train = create_graph_from_data(X_train_tensor, n_neighbors=15)  # 增加邻居数

# 定义超参数
in_feats = X_train_tensor.shape[1]
h_feats = 256  # 增大隐藏层维度
num_classes = len(np.unique(y_train_transf))
dropout = 0.5

# 创建GCN模型
model = GCN(in_feats=in_feats, hidden_feats=h_feats, out_feats=num_classes, dropout_rate=dropout)

# 优化器与学习率调度器
optimizer = optim.AdamW(model.parameters(), lr=0.005, weight_decay=1e-5)  # 使用AdamW
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

# 损失函数
loss_fn = torch.nn.CrossEntropyLoss()

# 训练过程
num_epochs = 100  # 增加训练轮数
train_losses = []
train_accuracies = []

for epoch in range(num_epochs):
    model.train()

    # 前向传播
    logits = model(G_train, G_train.ndata['feat'])

    # 计算损失
    loss = loss_fn(logits, y_train_tensor)

    # 反向传播和优化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # 更新学习率
    scheduler.step()

    # 记录训练损失
    train_losses.append(loss.item())

    # 计算训练精度
    _, train_preds = torch.max(logits, dim=1)
    train_accuracy = accuracy_score(y_train_tensor.cpu(), train_preds.cpu())
    train_accuracies.append(train_accuracy)

    # 每10轮打印一次损失和训练精度
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}, Train Accuracy: {train_accuracy:.4f}')

# 测试过程
model.eval()
with torch.no_grad():
    G_test = create_graph_from_data(X_test_tensor, n_neighbors=15)
    logits_test = model(G_test, G_test.ndata['feat'])
    predictions = torch.argmax(logits_test, dim=1)

    # 计算测试准确率
    accuracy = accuracy_score(y_test_transf, predictions.numpy())
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

/root/miniconda3/envs/GNNsIMPL/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


Epoch 0, Loss: 0.7000444531440735, Train Accuracy: 0.5379
Epoch 10, Loss: 0.640451967716217, Train Accuracy: 0.6206
Epoch 20, Loss: 0.4707907736301422, Train Accuracy: 0.8170
Epoch 30, Loss: 0.3156937062740326, Train Accuracy: 0.8891
Epoch 40, Loss: 0.22044765949249268, Train Accuracy: 0.9109
Epoch 50, Loss: 0.1253221482038498, Train Accuracy: 0.9546
Epoch 60, Loss: 0.0984925702214241, Train Accuracy: 0.9595
Epoch 70, Loss: 0.06610783189535141, Train Accuracy: 0.9828
Epoch 80, Loss: 0.0564689114689827, Train Accuracy: 0.9812
Epoch 90, Loss: 0.03838644176721573, Train Accuracy: 0.9868
Epoch 100, Loss: 0.03533089533448219, Train Accuracy: 0.9854
Epoch 110, Loss: 0.02958783507347107, Train Accuracy: 0.9925
Epoch 120, Loss: 0.024208160117268562, Train Accuracy: 0.9946
Epoch 130, Loss: 0.025786491110920906, Train Accuracy: 0.9928
Epoch 140, Loss: 0.02466065250337124, Train Accuracy: 0.9922
Epoch 150, Loss: 0.01867557503283024, Train Accuracy: 0.9952
Epoch 160, Loss: 0.018140053376555443, Tr

/root/miniconda3/envs/GNNsIMPL/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
